# **RADAR pipeline**
>RADAR (Rapid Analysis and Detection tool of Antimicrobial-Resistance)

---

This Colab notebook allows you to easily search the AMR genes within Whole Genome Sequencing (WGS) data using a RADAR (Rapid Analysis and Detection tool of Antimicrobial-Resistance) pipeline v1.0.

Please note that this Colab notebook is provided as an automatic pipeline allowing for an easy-to-use search tool for searching AMR genes.

---
**Citing this work**

Any publication that discloses findings arising from using this notebook should cite the BARDS paper.</br></br>

---
**More information**

You can find more information about how RADAR pipeline works in Github:

*   [SBML-KimsLab](https://github.com/SBL-Kimlab/radar)

---
**Reference**</br>

*   [USEARCH](https://drive5.com/usearch/) : *Edgar, R. C. (2010). Search and clustering orders of magnitude faster than BLAST. Bioinformatics, 26(19), 2460-2461.*</br>
*   [CIRCOS](http://circos.ca/) : *Krzywinski, M., Schein, J., Birol, I., Connors, J., Gascoyne, R., Horsman, D., ... & Marra, M. A. (2009). Circos: an information aesthetic for comparative genomics. Genome research, 19(9), 1639-1645.*


##  **1. PRE PROCESSING**(fin)


In [ ]:
#@title Install third-party software
#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left to download and import third-party software 
#@markdown in this Colab notebook. (See the [acknowledgements](https://github.com/SBL-Kimlab/radar) in our readme.)

#@markdown **Note**: This installs the software on the Colab 
#@markdown notebook in the cloud and not on your computer.
from IPython.utils import io
import os
import subprocess
import tqdm.notebook

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      !git clone https://github.com/SBL-Kimlab/radar.git
      print ( "\033[95m" + "\nRADAR pipeline: /content/radar/" )
      path_local = "/content/radar/"
      os.chdir( "/content/radar/" )
      pbar.update( 10 )
      #install biopython (Global library)
      %shell pip install biopython
      pbar.update( 10 )
      #update apt-get
      %shell apt-get update
      pbar.update( 6 )
      #install prodigal(Genome Annotation)
      %shell apt-get install prodigal
      pbar.update( 10 )
      #install libgd (Dependencies for Circos ideogram)
      %shell apt-get install libgd-dev
      pbar.update( 4 )
      #install cpan (Dependencies for Circos)
      %shell apt-get install cpanminus
      pbar.update( 20 )
      #install Circos (Visualization)
      %shell cpanm Clone Config::General Font::TTF::Font GD GD::Polyline Math::Bezier Math::Round Math::VecStat Params::Validate Readonly Regexp::Common SVG Set::IntSpan Statistics::Basic Text::Format
      pbar.update( 40 )
except subprocess.CalledProcessError:
  print(captured)
  raise

  0%|          | 0/100 [elapsed: 00:00 remaining: ?]

##  **2. RUN RADAR pipeline** (fin)

---
1. **DATABASE**: The RADAR pipeline can utilize the BARDS database and user-defined databases.
    * Two options are available. ***( 1.BARDS or 2.ETC )***
        *   **Option 1**: BARDS → Default DATABASE.
        *   **Option 2**: ETC → If you want to search using another user-defined (customized) database, ***please select "ETC" to upload the database***.

2. **SPECIES**: The entered SPECIES name is used as the name of the working folder during all analyses of the RADAR pipeline.
    * Uploaded WGS data is uploaded to the working folder of the entered SPECIES name and used as input for downstream analysis.
    * Example: escherichia


3. **BLAST_cutoff**: The RADAR pipeline can utilize the BARDS database and user-defined databases.
    * The BLAST cutoff can be easily adjusted by the user.
    * The blast cutoff provided as a slide bar has been scaled in increments of 0.5, and the entire blast searching result can be handled using the cutoff entered.

In [ ]:
#@title **2.RADAR pipeline** EXCUSION (fin)
#@markdown Please execute this cell by pressing the _Play_ button 

from IPython.utils import io
import os
import subprocess
import tqdm.notebook
from google.colab import files 


DATABASE = "BARDS" #@param ["BARDS", "ETC"]
path_db = path_local + "database/"
if DATABASE == "BARDS" : 
  DB = path_db + "%s.faa" % DATABASE
  print ( "Selected " + "\033[95m" + "BARDS\n" )
  print ( DB )
if DATABASE == "ETC" :  
  print ( "Selected " + "\033[95m" + "ETC\n" + "\033[91m" + "Upload your DATABASE" )
  uploaded = files.upload( ) 
  for fn in uploaded.keys() :
    print ( 'upload file "{name}" with length {length} bytes'.format( name = fn, length = len( uploaded[ fn ] ) ) )
    temp_db = "/content/radar/%s" %fn; DB = path_db + "USER_DB.faa" 
    !mv $temp_db $DB    

#print ( DB )
#include
file_include = path_local + "include/include.ipynb"
%run $file_include 
SPECIES = "escherichia" #@param [""] {allow-input: true}
pipeline = amr( SPECIES, DB )
path_pipeline = path_local + "pipeline/"
uploaded = files.upload( )
for fn in uploaded.keys() :
  print ( 'upload file "{name}" with length {length} bytes'.format( name = fn, length = len( uploaded[ fn ] ) ) )
  #from_file = "/content/%s" %fn; to_file = "/content/gdrive/MyDrive/radar/pipeline/antibiotics/genome/sequence/1.fna/%s/%s" %( STRAIN, fn )
  from_file = path_local + "%s" %fn; to_file = path_pipeline + "antibiotics/genome/sequence/1.fna/%s/%s" %( SPECIES, fn )
  !mv $from_file $to_file  

BLAST_cutoff = 0.85 #@param {type:"slider", min:0, max:1, step:0.05}
blast_cutoff = BLAST_cutoff
db = DB.split( "/" )[ -1 ]

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'

try:
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:      
      #change mode(usearch)
      file_usearch = path_local + "program/usearch"
      %shell chmod 777 $file_usearch
      #change mode(circos)
      file_circos = path_local + "program/circos/circos-0.69-6/bin/circos"
      %shell chmod 777 $file_circos
      pbar.update( 10 )
      
      #Annotation via prodigal software
      pipeline.method.prodigal( SPECIES )
      pbar.update( 10 )
      #DB compiling
      pipeline.method.db_statistics( SPECIES, db )
      pipeline.method.blast_method.udb_making( SPECIES, db )
      pbar.update( 10 )
      #BLAST RUN
      pipeline.method.blast_method.blastp_run( SPECIES, db )
      pbar.update( 25 )
      #BLAST parse & merge
      pipeline.method.blast_parse_method.blastp_parse( SPECIES, blast_cutoff )
      pipeline.method.blast_parse_method.blastp_merge( SPECIES, blast_cutoff )
      pbar.update( 25 )
      #Visualization
      pipeline.method.visual_method.make_config_files( SPECIES )
      pipeline.method.visual_method.run_circos( SPECIES )
      pbar.update( 20 )

except subprocess.CalledProcessError:
  print(captured)
  raise

Selected BARDS

/content/radar/database/BARDS.faa
Pipeline initiating...


Saving GCF_001007915.1_ASM100791v1_genomic.fna to GCF_001007915.1_ASM100791v1_genomic.fna
upload file "GCF_001007915.1_ASM100791v1_genomic.fna" with length 5355327 bytes


  0%|          | 0/100 [elapsed: 00:00 remaining: ?]